# Interactive simulation tutorial part 2: Simulation

This tutorial is intended to give an introduction to running simulations with Rational Speech Act (RSA) agents within the Lanag simulation framework. It is part of the Supplementary Information for *submitted*, which we will refer to as the *main paper*.

## 1. Loading libraries
First we need to load some libraries.

In [1]:
import $ivy.`com.markblokpoel::lanag-core:0.3.2`
interp.load.cp(new java.net.URL("https://github.com/markblokpoel/lanag-ambiguityhelps/blob/jupyternotebooks-restructuring/binaries/lanag-ambiguityhelps-0.1.jar?raw=true"))
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`

Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-almond_2.12/0.7.0/plotly-almond_2.12-0.7.0.pom
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-render_2.12/0.7.0/plotly-render_2.12-0.7.0.pom
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-core_2.12/0.7.0/plotly-core_2.12-0.7.0.pom
Downloaded https://repo1.maven.org/maven2/org/webjars/bower/plotly.js/1.41.3/plotly.js-1.41.3.pom
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-almond_2.12/0.7.0/plotly-almond_2.12-0.7.0.jar
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-core_2.12/0.7.0/plotly-core_2.12-0.7.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-render_2.12/0.7.0/plotly-render_2.12-0.7.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-almond_2.12/0.7.0/plotly-almond_2.12-0.7.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/webjars/bower/plotly.js/1.41.3/plotly.js-1.41.3-s

import $ivy.$                                   

import $ivy.$                                      

In [2]:
import com.markblokpoel.lanag.rsa.Lexicon
import com.markblokpoel.lanag.ambiguityhelps.RSA1ShotInteraction
import com.markblokpoel.lanag.ambiguityhelps.datastructures.InteractionData
import com.markblokpoel.lanag.ambiguityhelps.experiments.uniform.UniformPairGenerator
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import com.markblokpoel.lanag.rsa.Lexicon

import com.markblokpoel.lanag.ambiguityhelps.RSA1ShotInteraction

import com.markblokpoel.lanag.ambiguityhelps.datastructures.InteractionData

import com.markblokpoel.lanag.ambiguityhelps.experiments.uniform.UniformPairGenerator

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

### 1.1 Helper functions
Functions for working with notebooks (may be added to lanag core).

In [3]:
def lex2md(l: Lexicon): String = {
    val vs = l.vocabularySize
    val cs = l.contextSize
    
    var output = ""
    for(v <- -1 until vs) {
        for(c <- -1 until cs) {
            if(c < 0 && v < 0) output += "|&nbsp;|"
            else if (c < 0) output += "|$S_"+v+"$|"
            else if (v < 0) {
                output += "$C_"+c+"$|"
                if(c==cs-1) output += "\n"+ ((0 until 3).map(_ => "|--").mkString+"|")
            } else output += l(v,c)+"|"
        }
        output += "\n"
    }
    output
}

defined function lex2md

## 2. Generating agent pairs

Key to the study reported in the main paper was running simulations across a wide variety agent pairs that differ in lexicon ambiguity, the asymmetry between their lexicons and their order of pragmatic inference. Order has been explained in [part 1](./rsa-tutorial-part1.ipynb) of the tutorials. Here, we show how to work with the agent pair generator classes. There are three such classes, corresponding to the three different generation methods reported in the  main paper (viz. uniform ambiguity, implemented in `UniformPairGenerator`), and in the Supplementary Information (viz. Procedure I implemented in `RandomPairGenerator` and Procedure II implemented in `StructuredPairGenerator`).

We explain here `UniformPairGenerator`. The other generators are similarly structured. After completing this tutorial, you should be able to read the source code. To create an instance of `UniformPairGenerator` requires several parameters:

* `vocabularySize`, the number of signals $|V|$
* `contextSize`, the number of referents $|R|$
* `changeResolution`, a parameter between 0 and 1. Lower values create better coverage of the domain (cf. Figure 3 in the main paper).
* `sampleSize`, the number of samples generated per point in parameter space (this will become clear in a moment)
* `beta`, the beta parameter passed on to `RSA1ShotAgent`



In [4]:
val cpg = new UniformPairGenerator(
    vocabularySize = 8,
    contextSize = 4,
    changeResolution = 0.2,
    sampleSize = 10,
    beta = Double.PositiveInfinity
)

cpg: UniformPairGenerator = com.markblokpoel.lanag.ambiguityhelps.experiments.uniform.UniformPairGenerator@36de99dd

As explained in the main paper, pair generators work not by directly generating pairs of agents with a specific level of ambiguity and asymmetry. They create an proxy parameter space for which pairs of agents can be directly generated. Eventually, theses pairs of agents are grouped by ambiguity and asymmetry. We can access the parameterspace as follows using `.generateParameterSpace` which returns a `Vector` with all combinations of parameters. In this case, `ParametersUniform` consists of:
* `agent1Ambiguity`, the ambiguity of the lexicon of agent 1
* `agent2Ambiguity`, the ambiguity of the lexicon of agent 2 (only used in specific case)
* `changeRate`, the rate with which to change the lexicon of agent 1 to generate the lexicon of agent 2

What these parameters do, we will see momentarily. For now, we can observe how `changeResolution` affects the parameter space and how all levels of ambiguity (which depend on `contextSize`) are represented.

In [5]:
val paramSpace = cpg.generateParameterSpace

paramSpace: Seq[com.markblokpoel.lanag.ambiguityhelps.experiments.uniform.ParametersUniform] = Vector(
  ParametersUniform(1, 1, 0.0),
  ParametersUniform(1, 1, 0.2),
  ParametersUniform(1, 1, 0.4),
  ParametersUniform(1, 1, 0.6000000000000001),
  ParametersUniform(1, 1, 0.8),
  ParametersUniform(1, 1, 1.0),
  ParametersUniform(1, 2, 0.0),
  ParametersUniform(1, 2, 0.2),
  ParametersUniform(1, 2, 0.4),
  ParametersUniform(1, 2, 0.6000000000000001),
  ParametersUniform(1, 2, 0.8),
  ParametersUniform(1, 2, 1.0),
  ParametersUniform(1, 3, 0.0),
  ParametersUniform(1, 3, 0.2),
  ParametersUniform(1, 3, 0.4),
  ParametersUniform(1, 3, 0.6000000000000001),
  ParametersUniform(1, 3, 0.8),
  ParametersUniform(1, 3, 1.0),
  ParametersUniform(1, 4, 0.0),
  ParametersUniform(1, 4, 0.2),
  ParametersUniform(1, 4, 0.4),
  ParametersUniform(1, 4, 0.6000000000000001),
  ParametersUniform(1, 4, 0.8),
  ParametersUniform(1, 4, 1.0),
  ParametersUniform(2, 1, 0.0),
  ParametersUniform(2, 1, 0.2),
  Par

Using the function `generatePair` we can generate a pair of agents `AgentPair(agent1, agent2, originData)` for any point in the parameter space. `UniformPairGenerator` does this as follows:
1. For agent1, generate a random lexicon with specific ambiguity using `Lexicon.generateConsistentAmbiguityMapping(..)`
2. At random, select on of four methods to generate a lexicon for `agent2`:
  1. `removalBinaryMutation`, for each $s\in V$ add a proportion of signal-referent relations to the lexicon based on `changeRate`
  2. `additiveBinaryMutation`, for each $s\in V$ remove a proportion of signal-referent relations from the lexicon based on `changeRate`
  3. `mixReferents`, for each $s\in V$ switch a proportion of signal-referent relations around the lexicons central axis (i.e., `(i)(j) <=>(i)(contextSize-j)`).
  4. `generateConsistentAmbiguityMapping`, generates a lexicon for `agent2` from scratch
3. Several parameters pertaining to the generation for each pair of agents is stored in `originData` for later reference.

The next code chunk selects a random point in the parameter space and generates a pair of agents for it. We can inspect the origin data to see which of the four methods was selected and what the `changeRate` was set to. We can also compute the ambiguity and asymmetry.

In [6]:
val parameters = paramSpace(scala.util.Random.nextInt(paramSpace.size))

val pair = cpg.generatePair(parameters)

val agent1Ambiguity = pair.agent1.originalLexicon.meanAmbiguity()
val agent2Ambiguity = pair.agent2.originalLexicon.meanAmbiguity()
val asymmetry = pair.agent1.originalLexicon.asymmetryWith(pair.agent2.originalLexicon)

parameters: com.markblokpoel.lanag.ambiguityhelps.experiments.uniform.ParametersUniform = ParametersUniform(3, 3, 0.2)
pair: com.markblokpoel.lanag.core.AgentPair[com.markblokpoel.lanag.core.ReferentialIntention, com.markblokpoel.lanag.core.ContentSignal, com.markblokpoel.lanag.ambiguityhelps.RSA1ShotAgent, com.markblokpoel.lanag.ambiguityhelps.datastructures.OriginData] = AgentPair(
  Agent with order 0
0.0	1.0	1.0	1.0	
1.0	0.0	1.0	1.0	
1.0	1.0	1.0	0.0	
1.0	0.0	1.0	1.0	
1.0	1.0	1.0	0.0	
0.0	1.0	1.0	1.0	
1.0	1.0	1.0	0.0	
1.0	1.0	1.0	0.0	
,
  Agent with order 0
0.0	1.0	1.0	1.0	
1.0	0.0	1.0	1.0	
1.0	1.0	1.0	0.0	
1.0	0.0	1.0	1.0	
1.0	1.0	1.0	0.0	
0.0	1.0	1.0	1.0	
1.0	1.0	1.0	0.0	
1.0	1.0	1.0	0.0	
,
  OriginData(0.0, 0.2, NaN, NaN)
)
agent1Ambiguity: Double = 0.75
agent2Ambiguity: Double = 0.75
asymmetry: Double = 0.0

## 3. Running the simulation
The next code chunk demonstrators how we can use Scala's `map` and `flatMap` functions to setup the generation of `sampleSize` agent pairs per point in the parameter space. First, we transform (map) each point in the parameter space to a generator. The generators are iterators over `sampleSize` number of pairs in that point in parameter space.

In [7]:
val sampleGenerators = paramSpace.map(parameters => cpg.sampleGenerator(parameters))

sampleGenerators: Seq[Iterator[com.markblokpoel.lanag.core.AgentPair[com.markblokpoel.lanag.core.ReferentialIntention, com.markblokpoel.lanag.core.ContentSignal, com.markblokpoel.lanag.ambiguityhelps.RSA1ShotAgent, com.markblokpoel.lanag.ambiguityhelps.datastructures.OriginData]]] = Vector(
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  no

Then, we map each generator (for each point in the parameter space) to an iterator over interactions (for each point in the parameter space).

In [8]:
val interactions = sampleGenerators.map(generator => {
    // For each generator, we extract all pairs it generates and map those to a sequence of interactions.
    // One at order zero (default) and one at order 1. I.e., all agent pairs in the simulation interact
    // at multiple orders of pragmatic inference.
    generator.flatMap(pair => {
        val interaction = RSA1ShotInteraction(pair.agent1, pair.agent2, pair.originData, maxTurns=4)
        Seq(interaction, interaction.atOrder(1))
    })
})

interactions: Seq[Iterator[RSA1ShotInteraction]] = Vector(
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
  non-empty iterator,
...

Now that we have a large sequence of interactions covering as many points in the parameter space as possible, we can map each interaction to its concluding results by running the interaction through `runAndCollectData`. This returns a sequence of `RSA1InteractionData`, which contains logs for each interaction:
* `pairId`, a number identifying the pair of agents
* `agent1Order`, the order of agent 1
* `agent2Order`, the order of agent 2
* `agent1AmbiguityMean`, the mean ambiguity of agent 1's lexicon
* `agent1AmbiguityVar`, the mean ambiguity of agent 2's lexicon
* `agent2AmbiguityMean`, the variance of ambiguity of agent 1's lexicon
* `agent2AmbiguityVar`, the variance of ambiguity of agent 2's lexicon
* `asymmetry`, the asymmetry between the agents' lexicons
* `originData`, parameters used to generate the agents' lexicons
* `interaction`, a list containing logs of interaction between the agents

At this stage, the simulation is done and all that is left is to transform the data into a form that is suitable for analysis.

In [9]:
val data = interactions.flatMap(interaction => interaction.map(_.runAndCollectData))

data: Seq[InteractionData] = Vector(
  InteractionData(
    1L,
    0,
    0,
    1.0,
    0.0,
    1.0,
    0.0,
    0.0,
    OriginData(0.0, 0.0, NaN, NaN),
    List(
      TurnData(0, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
      TurnData(1, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
      TurnData(2, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
      TurnData(3, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0)))
    )
  ),
  InteractionData(
    1L,
    1,
    1,
    1.0,
    0.0,
    1.0,
    0.0,
    0.0,
    OriginData(0.0, 0.0, NaN, NaN),
    List(
      TurnData(0, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
      TurnData(1, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
      TurnData(2, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0))),
      TurnData(3, true, SpeakerData(Some(1.0)), ListenerData(Some(0.0)))
    )
  ),
  InteractionData(
    118L,
    0,
    0,
...

## 4. Summarizing the data
Since the main dependent measure is average communicative success of pairs of agents, we will need to summarize the agents' interactions in that manner. At the same time, we will flatten the hierarchical data structure returned in the previous step which enables us to export it to a flat `.csv` file.

In [12]:
import com.markblokpoel.lanag.ambiguityhelps.experiments.uniform.DataFlatUniform

val flatData = data.map(d => {
    DataFlatUniform(
          d.pairId,
          d.agent1Order,
          d.agent2Order,
          d.agent1AmbiguityMean,
          d.agent1AmbiguityVar,
          d.agent2AmbiguityMean,
          d.agent2AmbiguityVar,
          d.asymmetry,
          cpg.decodeChangeMethod(d.originData.parameter1),
          d.originData.parameter2,
          averageSuccess = d.interaction.count(i => i.success) / d.interaction.length.toDouble,
          averageEntropyAsSpeaker = d.interaction.foldLeft(0.0)((acc, e) =>
            acc + e.speakerData.speakerEntropy.getOrElse(0.0)) / d.interaction.length.toDouble,
          averageEntropyAsListener = d.interaction.foldLeft(0.0)((acc, e) =>
            acc + e.listenerData.listenerEntropy.getOrElse(0.0)) / d.interaction.length.toDouble
      )
})

import com.markblokpoel.lanag.ambiguityhelps.experiments.uniform.DataFlatUniform


flatData: Seq[DataFlatUniform] = Vector(
  DataFlatUniform(
    1L,
    0,
    0,
    1.0,
    0.0,
    1.0,
    0.0,
    0.0,
    "subtraction",
    0.0,
    1.0,
    1.0,
    0.0
  ),
  DataFlatUniform(
    1L,
    1,
    1,
    1.0,
    0.0,
    1.0,
    0.0,
    0.0,
    "subtraction",
    0.0,
    1.0,
    1.0,
    0.0
  ),
  DataFlatUniform(
    118L,
    0,
    0,
    1.0,
    0.0,
    1.0,
    0.0,
...

At this point, we can write each `DataFlatUniform` in `flatData` to a line in a `.csv` file, which is exactly what is done when using the software from command line. To display the results of our simulation in the notebook, we need to do some additional work.

In [14]:
val subset = flatData.filter(d => d.agent1AmbiguityMean == 3 && d.agent2AmbiguityMean == 3)

val xdata = subset.map(_.asymmetry).toSet.toList.sorted

val groups = subset.groupBy(d => {
  if(xdata.contains(d.asymmetry)) d.asymmetry
})

val xydata = for(x <- xdata) yield {
  x -> groups(x).map(pt => pt.averageSuccess).sum.toDouble / groups(x).size
}

val trace1 = Scatter(
    xydata.map(_._1).toSeq,
    xydata.map(_._2).toSeq,
    mode = ScatterMode(ScatterMode.Lines)
)
val plotData = Seq(trace1)

plot(plotData)

subset: Seq[DataFlatUniform] = Vector(
  DataFlatUniform(
    1441L,
    0,
    0,
    3.0,
    0.0,
    3.0,
    0.0,
    0.0,
    "addition",
    0.0,
    0.0,
    2.640560606055268,
    1.584962500721156
  ),
  DataFlatUniform(
    1441L,
    1,
    1,
    3.0,
    0.0,
    3.0,
    0.0,
    0.0,
    "addition",
    0.0,
    0.75,
    2.45263238406534,
    1.5762903670306987
  ),
  DataFlatUniform(
    1447L,
    0,
    0,
    3.0,
    0.0,
    3.0,
    0.0,
...
xdata: List[Double] = List(0.0, 0.125, 0.1875, 0.25, 0.3125, 0.375, 0.4375, 0.5)
groups: Map[AnyVal, Seq[DataFlatUniform]] = Map(
  0.0 -> Vector(
    DataFlatUniform(
      1441L,
      0,
      0,
      3.0,
      0.0,
      3.0,
      0.0,
      0.0,
      "addition",
      0.0,
      0.0,
      2.640560606055268,
      1.584962500721156
    ),
    DataFlatUniform(
      1441L,
      1,
      1,
      3.0,
      0.0,
      3.0,
      0.0,
      0.0,
      "addition",
      0.0,
      0.75,
      2.45263238406534,
      1.